In [1]:
import torch
import torch.distributions as dist

from daphne import daphne

#from primitives import funcprimitives #TODO
from tests import is_tol, run_prob_test,load_truth 

In [38]:
i = 3 
graph = [{'observe-data': ['fn', ['_', 'data', 'slope', 'bias'], ['let', ['xn', ['first', 'data']], ['let', ['yn', ['second', 'data']], ['let', ['zn', ['+', ['*', 'slope', 'xn'], 'bias']], ['let', ['dontcare9', ['observe', ['normal', 'zn', 1.0], 'yn']], ['rest', ['rest', 'data']]]]]]]}, {'V': ['observe3', 'observe6', 'observe4', 'observe7', 'sample2', 'sample1', 'observe8', 'observe5'], 'A': {'sample2': ['observe3', 'observe6', 'observe4', 'observe7', 'observe8', 'observe5'], 'sample1': ['observe3', 'observe6', 'observe4', 'observe7', 'observe8', 'observe5']}, 'P': {'sample1': ['sample*', ['normal', 0.0, 10.0]], 'sample2': ['sample*', ['normal', 0.0, 10.0]], 'observe3': ['observe*', ['normal', ['+', ['*', 'sample1', 1.0], 'sample2'], 1.0], 2.1], 'observe4': ['observe*', ['normal', ['+', ['*', 'sample1', 2.0], 'sample2'], 1.0], 3.9], 'observe5': ['observe*', ['normal', ['+', ['*', 'sample1', 3.0], 'sample2'], 1.0], 5.3], 'observe6': ['observe*', ['normal', ['+', ['*', 'sample1', 4.0], 'sample2'], 1.0], 7.7], 'observe7': ['observe*', ['normal', ['+', ['*', 'sample1', 5.0], 'sample2'], 1.0], 10.2], 'observe8': ['observe*', ['normal', ['+', ['*', 'sample1', 6.0], 'sample2'], 1.0], 12.9]}, 'Y': {'observe3': 2.1, 'observe4': 3.9, 'observe5': 5.3, 'observe6': 7.7, 'observe7': 10.2, 'observe8': 12.9}}, ['vector', 'sample1', 'sample2']]

print(graph)

[{'observe-data': ['fn', ['_', 'data', 'slope', 'bias'], ['let', ['xn', ['first', 'data']], ['let', ['yn', ['second', 'data']], ['let', ['zn', ['+', ['*', 'slope', 'xn'], 'bias']], ['let', ['dontcare9', ['observe', ['normal', 'zn', 1.0], 'yn']], ['rest', ['rest', 'data']]]]]]]}, {'V': ['observe3', 'observe6', 'observe4', 'observe7', 'sample2', 'sample1', 'observe8', 'observe5'], 'A': {'sample2': ['observe3', 'observe6', 'observe4', 'observe7', 'observe8', 'observe5'], 'sample1': ['observe3', 'observe6', 'observe4', 'observe7', 'observe8', 'observe5']}, 'P': {'sample1': ['sample*', ['normal', 0.0, 10.0]], 'sample2': ['sample*', ['normal', 0.0, 10.0]], 'observe3': ['observe*', ['normal', ['+', ['*', 'sample1', 1.0], 'sample2'], 1.0], 2.1], 'observe4': ['observe*', ['normal', ['+', ['*', 'sample1', 2.0], 'sample2'], 1.0], 3.9], 'observe5': ['observe*', ['normal', ['+', ['*', 'sample1', 3.0], 'sample2'], 1.0], 5.3], 'observe6': ['observe*', ['normal', ['+', ['*', 'sample1', 4.0], 'sample2'

In [39]:
fn_defs = graph[0]
V = graph[1]['V']
A = graph[1]['A']
P = graph[1]['P']
Y = graph[1]['Y']
ret_vals = graph[2]

In [40]:
print(V)

['observe3', 'observe6', 'observe4', 'observe7', 'sample2', 'sample1', 'observe8', 'observe5']


In [45]:
def topological_sort(A, V):
    visited = {v:False for v in V}
    stack = []
    
    for v in V:
        if visited[v] == False:
            topo_sort_util(v, A, V, visited, stack)
            
    return stack

def topo_sort_util(v, A, V, visited, stack):
    
    visited[v] = True
    
    if v in A:
        for adj_v in A[v]:
            if visited[adj_v] == False:
                topo_sort_util(adj_v, A, V, visited, stack)
            
    stack.insert(0, v)
    

In [46]:
print(topological_sort(A, V))

['sample1', 'sample2', 'observe5', 'observe8', 'observe7', 'observe4', 'observe6', 'observe3']


In [48]:
sorted_V = topological_sort(A, V)

for v in sorted_V:
    print(v)
    print(P[v])
    print("\n\n")

sample1
['sample*', ['normal', 0.0, 10.0]]



sample2
['sample*', ['normal', 0.0, 10.0]]



observe5
['observe*', ['normal', ['+', ['*', 'sample1', 3.0], 'sample2'], 1.0], 5.3]



observe8
['observe*', ['normal', ['+', ['*', 'sample1', 6.0], 'sample2'], 1.0], 12.9]



observe7
['observe*', ['normal', ['+', ['*', 'sample1', 5.0], 'sample2'], 1.0], 10.2]



observe4
['observe*', ['normal', ['+', ['*', 'sample1', 2.0], 'sample2'], 1.0], 3.9]



observe6
['observe*', ['normal', ['+', ['*', 'sample1', 4.0], 'sample2'], 1.0], 7.7]



observe3
['observe*', ['normal', ['+', ['*', 'sample1', 1.0], 'sample2'], 1.0], 2.1]





In [52]:
for item in fn_defs.items():
    print(item[0])
    print(item[1])

observe-data
['fn', ['_', 'data', 'slope', 'bias'], ['let', ['xn', ['first', 'data']], ['let', ['yn', ['second', 'data']], ['let', ['zn', ['+', ['*', 'slope', 'xn'], 'bias']], ['let', ['dontcare9', ['observe', ['normal', 'zn', 1.0], 'yn']], ['rest', ['rest', 'data']]]]]]]


In [53]:
i = 2
ast = daphne(['desugar', '-i', '../CS532-HW2/programs/{}.daphne'.format(i)])
print(ast)

[['defn', 'observe-data', ['_', 'data', 'slope', 'bias'], ['let', ['xn', ['first', 'data']], ['let', ['yn', ['second', 'data']], ['let', ['zn', ['+', ['*', 'slope', 'xn'], 'bias']], ['let', ['dontcare0', ['observe', ['normal', 'zn', 1.0], 'yn']], ['rest', ['rest', 'data']]]]]]], ['let', ['slope', ['sample', ['normal', 0.0, 10.0]]], ['let', ['bias', ['sample', ['normal', 0.0, 10.0]]], ['let', ['data', ['vector', 1.0, 2.1, 2.0, 3.9, 3.0, 5.3, 4.0, 7.7, 5.0, 10.2, 6.0, 12.9]], ['let', ['dontcare1', ['let', ['a2', 'slope'], ['let', ['a3', 'bias'], ['let', ['acc4', ['observe-data', 0, 'data', 'a2', 'a3']], ['let', ['acc5', ['observe-data', 1, 'acc4', 'a2', 'a3']], ['let', ['acc6', ['observe-data', 2, 'acc5', 'a2', 'a3']], ['let', ['acc7', ['observe-data', 3, 'acc6', 'a2', 'a3']], ['let', ['acc8', ['observe-data', 4, 'acc7', 'a2', 'a3']], ['let', ['acc9', ['observe-data', 5, 'acc8', 'a2', 'a3']], 'acc9']]]]]]]]], ['vector', 'slope', 'bias']]]]]]


In [55]:
defn_ast = ast[0]

In [56]:
print(defn_ast)

['defn', 'observe-data', ['_', 'data', 'slope', 'bias'], ['let', ['xn', ['first', 'data']], ['let', ['yn', ['second', 'data']], ['let', ['zn', ['+', ['*', 'slope', 'xn'], 'bias']], ['let', ['dontcare0', ['observe', ['normal', 'zn', 1.0], 'yn']], ['rest', ['rest', 'data']]]]]]]


In [57]:
defn_ast[1]

'observe-data'

In [58]:
defn_ast[2]

['_', 'data', 'slope', 'bias']

In [60]:
defn_ast[3]

['let',
 ['xn', ['first', 'data']],
 ['let',
  ['yn', ['second', 'data']],
  ['let',
   ['zn', ['+', ['*', 'slope', 'xn'], 'bias']],
   ['let',
    ['dontcare0', ['observe', ['normal', 'zn', 1.0], 'yn']],
    ['rest', ['rest', 'data']]]]]]

In [62]:
for item in fn_defs.items():
    defn_graph = item

print(defn_graph)

('observe-data', ['fn', ['_', 'data', 'slope', 'bias'], ['let', ['xn', ['first', 'data']], ['let', ['yn', ['second', 'data']], ['let', ['zn', ['+', ['*', 'slope', 'xn'], 'bias']], ['let', ['dontcare9', ['observe', ['normal', 'zn', 1.0], 'yn']], ['rest', ['rest', 'data']]]]]]])


In [63]:
len(defn_graph)

2

In [65]:
defn_graph[0]

'observe-data'

In [71]:
defn_graph[1][2]

['let',
 ['xn', ['first', 'data']],
 ['let',
  ['yn', ['second', 'data']],
  ['let',
   ['zn', ['+', ['*', 'slope', 'xn'], 'bias']],
   ['let',
    ['dontcare9', ['observe', ['normal', 'zn', 1.0], 'yn']],
    ['rest', ['rest', 'data']]]]]]

In [72]:
print(P)

{'sample1': ['sample*', ['normal', 0.0, 10.0]], 'sample2': ['sample*', ['normal', 0.0, 10.0]], 'observe3': ['observe*', ['normal', ['+', ['*', 'sample1', 1.0], 'sample2'], 1.0], 2.1], 'observe4': ['observe*', ['normal', ['+', ['*', 'sample1', 2.0], 'sample2'], 1.0], 3.9], 'observe5': ['observe*', ['normal', ['+', ['*', 'sample1', 3.0], 'sample2'], 1.0], 5.3], 'observe6': ['observe*', ['normal', ['+', ['*', 'sample1', 4.0], 'sample2'], 1.0], 7.7], 'observe7': ['observe*', ['normal', ['+', ['*', 'sample1', 5.0], 'sample2'], 1.0], 10.2], 'observe8': ['observe*', ['normal', ['+', ['*', 'sample1', 6.0], 'sample2'], 1.0], 12.9]}


In [86]:
i = 2
task, expr = P[sorted_V[i]][0], P[sorted_V[i]][1]
print(sorted_V[i])
print(task)
print(expr)
print(Y[sorted_V[i]])

observe5
observe*
['normal', ['+', ['*', 'sample1', 3.0], 'sample2'], 1.0]
5.3


In [80]:
task == "observe*"

True

In [87]:
print(ret_vals)

['vector', 'sample1', 'sample2']


In [83]:
expr[1]

['+', ['*', 'sample1', 3.0], 'sample2']

# Test Run

In [88]:
print(graph)

[{'observe-data': ['fn', ['_', 'data', 'slope', 'bias'], ['let', ['xn', ['first', 'data']], ['let', ['yn', ['second', 'data']], ['let', ['zn', ['+', ['*', 'slope', 'xn'], 'bias']], ['let', ['dontcare9', ['observe', ['normal', 'zn', 1.0], 'yn']], ['rest', ['rest', 'data']]]]]]]}, {'V': ['observe3', 'observe6', 'observe4', 'observe7', 'sample2', 'sample1', 'observe8', 'observe5'], 'A': {'sample2': ['observe3', 'observe6', 'observe4', 'observe7', 'observe8', 'observe5'], 'sample1': ['observe3', 'observe6', 'observe4', 'observe7', 'observe8', 'observe5']}, 'P': {'sample1': ['sample*', ['normal', 0.0, 10.0]], 'sample2': ['sample*', ['normal', 0.0, 10.0]], 'observe3': ['observe*', ['normal', ['+', ['*', 'sample1', 1.0], 'sample2'], 1.0], 2.1], 'observe4': ['observe*', ['normal', ['+', ['*', 'sample1', 2.0], 'sample2'], 1.0], 3.9], 'observe5': ['observe*', ['normal', ['+', ['*', 'sample1', 3.0], 'sample2'], 1.0], 5.3], 'observe6': ['observe*', ['normal', ['+', ['*', 'sample1', 4.0], 'sample2'

In [89]:
fn_defs = graph[0]
V = graph[1]['V']
A = graph[1]['A']
P = graph[1]['P']
Y = graph[1]['Y']
ret_vals = graph[2]

In [90]:
import torch
import math
import operator as op

def pure_hashmap_update(d, k, v):
    if isinstance(k, torch.Tensor):
        k = k.item()
    d2 = d.copy()
    d2.update({k:v})
    return d2

# got a tip from Masoud on this
def vector_list_creation(args):
    try:
        return torch.stack(args)
    except:
        return args

def mat_transpose(args):
    if isinstance(args, tuple):
        if len(args) == 1:
            args = args[0]
        else:
            assert False, "Multi-element tuple???"
    try:
        return args.t()
    except:
        assert False, "whoops"

def mat_repmat(t, i1, i2):
    return t.repeat(i1.long().item(), i2.long().item())

# inspired by https://norvig.com/lispy.html
def eval_env():
    env = {}
    env.update({
        '+': torch.add,
        '-': torch.sub,
        '*': torch.mul,
        '/': torch.div,
        '>': torch.gt,
        '<': torch.lt,
        '>=': torch.ge,
        '<=': torch.le, 
        '=': torch.eq,
        'sqrt': torch.sqrt,
        'exp': torch.exp,
        'mat-tanh': torch.tanh,
        'mat-add': torch.add,
        'mat-mul': torch.matmul,
        'mat-repmat': mat_repmat,
        'mat-transpose': lambda *x: mat_transpose(x),
        'vector': lambda *x: vector_list_creation(x),
        'hash-map': lambda *x : dict(zip([i.item() if isinstance(i, torch.Tensor) else i for i in x[::2]], x[1::2])),
        'get': lambda x, y: x[y.long()] if isinstance(x, torch.Tensor) else x[y.item() if isinstance(y, torch.Tensor) else y],
        'put': lambda x, y, z: torch.cat((x[:y.long()], torch.tensor([z]), x[y.long()+1:])) if isinstance(x, torch.Tensor) else pure_hashmap_update(x,y,z),
        'append' : lambda x, y: torch.cat((x, torch.tensor([y]))),
        'first' : lambda x: x[0],
        'second': lambda x: x[1],
        'rest': lambda x: x[1:],
        'last' : lambda x: x[-1],
        'remove': lambda x, y : torch.cat((x[:y.long()], x[y.long()+1:])) if isinstance(x, torch.Tensor) else {i:x[i] for i in x if i != y},
        'normal': torch.distributions.Normal,
        'beta': torch.distributions.beta.Beta,
        'exponential': torch.distributions.exponential.Exponential,
        'uniform': torch.distributions.uniform.Uniform,
        'bernoulli': torch.distributions.bernoulli.Bernoulli,
        'discrete': lambda *x: torch.distributions.categorical.Categorical(x[0]) 
        })


    return env


In [97]:
def deterministic_evaluate(e, l, sig=None):
    # variable reference OR procedure OR just a string
    if isinstance(e, str):        
        # global procedures take precedence over locally defined vars
        if e in ENV:
            return ENV[e], sig
        elif e in l:
            return l[e], sig
        # could allow for hashmaps with string keys; for debugging setting this to fail
        else:
            assert False, "Unknown symbol: {}".format(e)
    # constant number
    elif isinstance(e, (int, float)):   
        return torch.tensor(float(e)), sig
    # if statements
    elif e[0] == 'if':
        (_, test, conseq, alt) = e
        exp = (conseq if deterministic_evaluate(test, l)[0] else alt)
        return deterministic_evaluate(exp, l)
    # let statements
    elif e[0] == 'let':
        # get symbol
        symbol = e[1][0]
        # get value of e1
        value, _ = deterministic_evaluate(e[1][1], l)
        # evaluate e2 with value 
        return deterministic_evaluate(e[2], {**l, symbol: value})
    # sample statement
    if e[0] == 'sample':
        dist = deterministic_evaluate(e[1], l)[0]
        # make sure it is a distribution object
        assert getattr(dist, '__module__', None).split('.')[:2] == ['torch', 'distributions']
        return dist.sample(), sig
    # obsere statements
    # TODO: change this, maybe in this hw or for hw3
    if e[0] == 'observe':
        dist = deterministic_evaluate(e[1], l)[0] # get dist
        y = deterministic_evaluate(e[2], l)[0]    # get observed value
        # make sure it is a distribution object
        assert getattr(dist, '__module__', None).split('.')[:2] == ['torch', 'distributions']
        # TODO: do something with observed value
        return dist.sample(), sig
    # procedure call, either primitive or user-defined
    else:
        result = deterministic_evaluate(e[0], l)
        proc, sig = result
        # primitives are functions
        if callable(proc):
            args = [deterministic_evaluate(arg, l)[0] for arg in e[1:]]
            result, sig = proc(*args), sig
            return result, sig
        # user defined functions are not
        else:
            # as written in algorithm 6
            v_is, e0 = proc 
            assert(len(v_is) == len(e[1:]))
            c_is = [deterministic_evaluate(arg, l)[0] for arg in e[1:]]
            l_proc = dict(zip(v_is, c_is))
            return deterministic_evaluate(e0, {**l, **l_proc})

In [92]:
ENV = eval_env()

In [93]:
for defn in fn_defs.items():
    f_name = defn[0]
    f_v_is = defn[1][1]
    f_expr = defn[1][2]
    ENV.update({f_name: (f_v_is, f_expr)})

In [94]:
print(ENV)

{'+': <built-in method add of type object at 0x10e802c90>, '-': <built-in method sub of type object at 0x10e802c90>, '*': <built-in method mul of type object at 0x10e802c90>, '/': <built-in method div of type object at 0x10e802c90>, '>': <built-in method gt of type object at 0x10e802c90>, '<': <built-in method lt of type object at 0x10e802c90>, '>=': <built-in method ge of type object at 0x10e802c90>, '<=': <built-in method le of type object at 0x10e802c90>, '=': <built-in method eq of type object at 0x10e802c90>, 'sqrt': <built-in method sqrt of type object at 0x10e802c90>, 'exp': <built-in method exp of type object at 0x10e802c90>, 'mat-tanh': <built-in method tanh of type object at 0x10e802c90>, 'mat-add': <built-in method add of type object at 0x10e802c90>, 'mat-mul': <built-in method matmul of type object at 0x10e802c90>, 'mat-repmat': <function mat_repmat at 0x7ff0e360a1f0>, 'mat-transpose': <function eval_env.<locals>.<lambda> at 0x7ff0e360aa60>, 'vector': <function eval_env.<lo

In [95]:
sorted_V = topological_sort(A, V)

In [101]:
l = {}
for v in sorted_V:
    task, expr = P[v][0], P[v][1]
    if task == "sample*":
        dist, _ = deterministic_evaluate(expr, l)
        l.update({v: dist.sample()})
    # TODO: for now treat observes like samples; fix this later
    if task == "observe*":
        dist, _ = deterministic_evaluate(expr, l)
        l.update({v: dist.sample()})

In [102]:
l

{'sample1': tensor(24.1250),
 'sample2': tensor(13.2892),
 'observe5': tensor(86.2690),
 'observe8': tensor(158.7638),
 'observe7': tensor(134.0264),
 'observe4': tensor(60.4891),
 'observe6': tensor(109.4864),
 'observe3': tensor(37.4515)}

In [104]:
deterministic_evaluate(ret_vals, l)[0]

tensor([24.1250, 13.2892])